In [1]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import torch

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.structures import BoxMode

In [2]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    i = 0
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        i+=1
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["image_id"] = i
        record["height"] = 1080
        record["width"] = 1920

      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                #"id": i,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [3]:
classes = ['asphalt', 'tile', 'snowy_dirt', 'snow', 'dirty_tile', 'icy_tile']

data_path = 'DATA/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes, thing_colors = [
        (255,255,0),(0,0,255),(0,255,0),(255,0,255), (180,165,0), (187,132,156)]) #Done by me
  
    

microcontroller_metadata = MetadataCatalog.get("category_train")

In [4]:
cfg_instance_seg = get_cfg()
cfg_instance_seg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_instance_seg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
cfg_instance_seg.MODEL.WEIGHTS = os.path.join("output", "model_final.pth")
cfg_instance_seg.DATASETS.TEST = ("category_test")
cfg_instance_seg.MODEL.ROI_HEADS.NUM_CLASSES = 6
predictor = DefaultPredictor(cfg_instance_seg)
evaluator = COCOEvaluator("category_test", cfg_instance_seg, False, output_dir="./output/eval/")

[11/26 17:02:06 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

WARNING [11/26 17:02:06 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[11/26 17:02:06 d2.evaluation.coco_evaluation]: Trying to convert 'category_test' to COCO format ...
[11/26 17:02:06 d2.data.datasets.coco]: Converting annotations of dataset 'category_test' to COCO format ...)
[11/26 17:02:06 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[11/26 17:02:06 d2.data.datasets.coco]: Conversion finished, #images: 8, #annotations: 30
[11/26 17:02:06 d2.data.datasets.coco]: Caching COCO format annotations at './output/eval/category_test_coco_format.json' ...


In [5]:
val_loader = build_detection_test_loader(cfg_instance_seg, "category_test")
inference_on_dataset(predictor.model, val_loader, evaluator)
#evaluator.evaluate()

[11/26 17:02:06 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|  asphalt   | 9            |    tile    | 6            | snowy_dirt | 6            |
|    snow    | 5            | dirty_tile | 2            |  icy_tile  | 2            |
|            |              |            |              |            |              |
|   total    | 30           |            |              |            |              |
[11/26 17:02:06 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/26 17:02:06 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common.NumpySerializedList'>
[11/26 17:02:06 d2.data.common]: Serializing 8 elements to byte tensors and concatenating them all ...
[11

OrderedDict([('bbox',
              {'AP': 35.88251087013463,
               'AP50': 55.91911572109591,
               'AP75': 38.6033841479386,
               'APs': nan,
               'APm': 0.0,
               'APl': 38.17230473047305,
               'AP-asphalt': 57.478233537639476,
               'AP-tile': 57.79702970297029,
               'AP-snowy_dirt': 18.435643564356436,
               'AP-snow': 31.08910891089109,
               'AP-dirty_tile': 15.148514851485148,
               'AP-icy_tile': 35.34653465346535}),
             ('segm',
              {'AP': 30.21040913615171,
               'AP50': 41.40866467599141,
               'AP75': 36.379066478076375,
               'APs': nan,
               'APm': 0.0,
               'APl': 32.365236523652364,
               'AP-asphalt': 52.83011158258684,
               'AP-tile': 62.32673267326733,
               'AP-snowy_dirt': 7.2937293729372925,
               'AP-snow': 28.51485148514851,
               'AP-dirty_tile': 0